In [ ]:
#!pip install ase
import numpy as np
import warnings
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from ase.io import read
import IPython.display as display
from IPython.display import HTML
warnings.filterwarnings("ignore",category=RuntimeWarning)

#### $V(r) = 4\epsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^{6} \right] $
#### $F(r)= -\frac{dV}{dr}$
##### $  \frac{d}{dr}\left(\frac{\sigma}{r}\right)^{12} = \frac{d}{dr} \left( \sigma^{12} r^{-12} \right) = -12 \sigma^{12} r^{-13} = -12 \left(\frac{\sigma}{r}\right)^{12} \frac{1}{r} \ $
#### $\frac{d}{dr}\left(\frac{\sigma}{r}\right)^6 = \frac{d}{dr} \left( \sigma^6 r^{-6} \right) = -6 \sigma^6 r^{-7} = -6 \left(\frac{\sigma}{r}\right)^6 \frac{1}{r} $
#### $\frac{dV}{dr} = 4 \epsilon \left[ -12 \left(\frac{\sigma}{r}\right)^{12} \frac{1}{r} + 6 \left(\frac{\sigma}{r}\right)^6 \frac{1}{r} \right]$
#### $\frac{dV}{dr} = -48 \epsilon \left(\frac{\sigma}{r}\right)^{12} \frac{1}{r} + 24 \epsilon \left(\frac{\sigma}{r}\right)^6 \frac{1}{r} $
#### $ F(r) = \frac{48 \epsilon \sigma^{12}}{r^{13}} - \frac{24 \epsilon \sigma^6}{r^{7}}$
#### $F(r) = \frac{24\epsilon}{r} \left[ 2\left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6 \right]$
#### $F(r) = \frac{24\epsilon}{r} \left[ 2\left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^{6} \right]$

# Task 1
## Define a function to calculate LJ potential and forces 

In [ ]:
# Function to compute the Lennard-Jones potential and force
def lj_potential_and_force(r, sigma, epsilon):
    sr6 = (sigma / r) ** 6
    sr12 = sr6 ** 2
    potential = 4 * epsilon * (sr12 - sr6)
    force = 24 * epsilon * (2 * sr12 - sr6) / r
    return potential, force

# Task 2
## Define a function to calculate to apply Periodic Boundary Condition

In [ ]:
def pbc(position1, position2, box_length):
    r_diff = position1 - position2
    for i in range(len(r_diff)):
        if r_diff[i] > 0.5 * box_length:
            r_diff[i] -= box_length
        elif r_diff[i] < -0.5 * box_length:
            r_diff[i] += box_length
    return r_diff

# Task 3
## Calculate total potential energy and forces

In [ ]:
# Function to compute the total potential energy and forces
def compute_forces(positions, num_particles, box_length, sigma, epsilon, r_cut, mass):
    accelerations = np.zeros_like(positions)
    potential_energy = 0.0
    for i in range(num_particles):
        for j in range(i + 1, num_particles):
            rij = pbc(positions[i], positions[j], box_length)
            r = np.sqrt(np.dot(rij, rij))
            if r < r_cut:
                potential, force = lj_potential_and_force(r, sigma, epsilon)
                potential_energy += potential
                force_vector = (force * rij / r) / mass
                accelerations[i] += force_vector
                accelerations[j] -= force_vector
    return potential_energy, accelerations


## Function to read and write file

In [ ]:
# Function to read XYZ file
def read_xyz(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    num_particles = int(lines[0].strip())
    positions = []
    for line in lines[2:2+num_particles]:
        _, x, y, z = line.split()
        positions.append([float(x), float(y), float(z)])
    
    positions = np.array(positions)
    return positions, num_particles

# Function to write XYZ file
def write_xyz(file_path, positions, step):
    with open(file_path, 'a') as f:
        f.write(f"{len(positions)}\n")
        f.write(f"Step {step}\n")
        for pos in positions:
            f.write(f"Ar {pos[0]} {pos[1]} {pos[2]}\n")

# Function to write energy data
def write_energy(file_path, step, potential_energy, kinetic_energy, total_energy):
    with open(file_path, 'a') as f:
        f.write(f"Step {step}, Potential Energy: {potential_energy:.4f}, Kinetic Energy: {kinetic_energy:.4f}, Total Energy: {total_energy:.4f}\n")

### Velocity Verlet Integration Method

1. **Update Positions**:

   $ \mathbf{r}(t + \Delta t) = \mathbf{r}(t) + \mathbf{v}(t) \Delta t + \frac{1}{2} \mathbf{a}(t) \Delta t^2 $


2. **Compute Forces and Update Accelerations**:

3. **Update Velocities**:

   $ \mathbf{v}(t + \Delta t) = \mathbf{v}(t) + \frac{1}{2} \left(\mathbf{a}(t) + \mathbf{a}(t + \Delta t)\right) \Delta t $




# Task 4
## Define velocity verlet integrator

In [ ]:
# Velocity Verlet integration
def velocity_verlet(positions, velocities, accelerations, num_particles, box_length, time_step, sigma, epsilon, r_cut, mass):
    # Update positions
    positions += velocities * time_step + 0.5 * accelerations * time_step ** 2
    positions = positions % box_length  # Apply periodic boundary conditions
    
    # Compute new accelerations and potential energy
    potential_energy, new_accelerations = compute_forces(positions, num_particles, box_length, sigma, epsilon, r_cut, mass)
    
    # Update velocities
    velocities += 0.5 * (accelerations + new_accelerations) * time_step
    
    return positions, velocities, new_accelerations, potential_energy


In [ ]:
#Parameter Initialization
sigma = 3.4  # in Angstroms
epsilon = 0.238  # in kcal/mol

# Simulation parameters
r_cut = 2.5 * sigma  # Cutoff distance, scaled by sigma
kB = 1.9872041e-3  # Boltzmann constant in kcal/(mol*K)

# MD simulation parameters
box_length = 10.0  # Angstroms
time_step = 0.001  # Picoseconds
num_steps = 1000  # Total number of steps
temperature = 300.0  # Kelvin
m_argon = 39.948  # atomic mass units (amu)


In [ ]:
#Initialize velocities
positions, num_particles = read_xyz('argons.xyz')
velocities = np.random.randn(num_particles, 3)
velocities -= np.mean(velocities, axis=0)  # Zero total momentum
velocities *= np.sqrt(3 * kB * temperature / epsilon) / np.sqrt(m_argon)  # Scale velocities
# Output files
xyz_output_file = 'md_trajectory.xyz'
energy_output_file = 'md_energy.txt'

# Clear the output files if they already exist
open(xyz_output_file, 'w').close()
open(energy_output_file, 'w').close()

# Task 5
## Run MD simulation

In [ ]:

accelerations = np.zeros_like(positions)
potential_energy, accelerations = compute_forces(positions, num_particles, box_length, sigma, epsilon, r_cut, m_argon)

for step in range(num_steps):
    positions, velocities, accelerations, potential_energy = velocity_verlet(
        positions, velocities, accelerations, num_particles, box_length, time_step, sigma, epsilon, r_cut, m_argon)
    kinetic_energy = 0.5 * np.sum(velocities ** 2) * m_argon 
    total_energy = potential_energy + kinetic_energy
    
    # Write the positions to the XYZ file
    write_xyz(xyz_output_file, positions, step)
    
    # Write the energy data to the text file
    write_energy(energy_output_file, step, potential_energy, kinetic_energy, total_energy)
    
    if step % 10 == 0:
        print(f"Step {step}/{num_steps}, Total Energy: {total_energy:.4f}")

## Visualize Trajectory

In [ ]:
plt.ion()

# Function to read the XYZ file
def read_xyz_file(filename):
    atoms = read(filename, index=':')  # Reads all frames
    return atoms

def update(num, atoms, graph, ax):
    ax.clear()  # Clear previous frame
    ax.set_xlim([atoms[num].positions[:, 0].min() - 1, atoms[num].positions[:, 0].max() + 1])
    ax.set_ylim([atoms[num].positions[:, 1].min() - 1, atoms[num].positions[:, 1].max() + 1])
    ax.set_zlim([atoms[num].positions[:, 2].min() - 1, atoms[num].positions[:, 2].max() + 1])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    graph = ax.scatter(atoms[num].positions[:, 0], atoms[num].positions[:, 1], atoms[num].positions[:, 2], c='blue', marker='o')
    return graph,

filename = 'md_trajectory.xyz'

# Read the XYZ file
trajectory = read_xyz_file(filename)

# Setup the figure and axis for 3D plotting
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Create animation
ani = FuncAnimation(fig, update, frames=len(trajectory), fargs=(trajectory, None, ax), blit=False)
html = HTML(ani.to_jshtml())
display.display(html)